## MOEX API
В данном ноутбуке исследуем функционал API Московской фондовой биржи. Цели:
1. Разобраться с основными функциями API (+)
1. Научиться запрашивать и выгружать информацию в csv о тикерах (+)
1. Научиться запрашивать и выгружать в csv информацию о котировках (+)

## Основные функции API

Информацию о работе API можно найти по ссылке (https://www.moex.com/a2193). Там же можно скачать pdf с базовым описанием работы API. Описание всех функций можно найти по ссылке (http://iss.moex.com/iss/reference/).



In [1]:
import requests
import json
import pandas as pd
import time

url_prefix = 'http://iss.moex.com/'

Получим информацию о торгуемых бумагах на московской бирже, а также получем спецификацию по одной из них.

In [3]:
response = requests.get(url_prefix + '/iss/securities.json')
json_data = json.loads(response.text)
data = json_data['securities']['data']
data_cols = json_data['securities']['columns']
df_securities = pd.DataFrame(columns=data_cols)
s = 0

while len(data)>0:
    params = {'start': str(s),
              'limit': '100'}
    response = requests.get(url_prefix + '/iss/securities.json', params)
    json_data = json.loads(response.text)
    data = json_data['securities']['data']
    if len(data)>0:
        df_securities = df_securities.append(pd.DataFrame(data=data, columns=data_cols))
    s = s + 100
    #sleep

df_securities.columns = json_data['securities']['columns']
df_securities.reset_index(inplace=True)
df_securities.drop('index', axis=1, inplace=True)
df_securities.head(10)

,id,secid,shortname,regnumber,name,isin,is_traded,emitent_id,emitent_title,emitent_inn,emitent_okpo,gosreg,type,group,primary_boardid,marketprice_boardid
0,154676,AAPL,Apple,None,Apple Inc.,US0378331005,1,1.281e+06,Apple Inc,None,None,None,common_share,stock_shares,EQRD,None
1,12441,ABRD,АбрауДюрсо,1-02-12500-A,Абрау-Дюрсо ПАО ао,RU000A0JS5T7,1,2556,"Публичное акционерное общество ""Абрау – Дюрсо""",7727620673,81521198,1-02-12500-A,common_share,stock_shares,TQBR,TQBR
2,131568,ACKO,АСКО ао,1-01-52065-Z,АСКО-СТРАХОВАНИЕ ПАО ао,RU000A0JXS91,1,1.09870e+06,"Публичное акционерное общество ""АСКО-СТРАХОВАНИЕ""",7453297458,31212003,1-01-52065-Z,common_share,stock_shares,TQDE,TQDE
3,85909,acru,"ОАО АК ""Рубин""",1-03-01692-A,"Открытое акционерное общество ""Авиационная кор...",RU000A0JRM20,1,486957,"публичное акционерное общество ""Авиационная ко...",5001000034,None,1-03-01692-A,common_share,stock_shares,MXBD,None
4,139931163,AFH0,AFLT-3.20,None,Фьючерсный контракт AFLT-3.20,None,1,1300,"Публичное акционерное общество ""Аэрофлот – рос...",7712040126,29063984,None,futures,futures_forts,RFUD,None
5,2699,AFKS,Система ао,1-05-01669-A,"АФК ""Система"" ПАО ао",RU000A0DQZE3,1,2046,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""АКЦИОНЕРНАЯ ФИ...",7703104630,27987276,1-05-01669-A,common_share,stock_shares,TQBR,TQBR
6,2700,AFLT,Аэрофлот,1-01-00010-A,Аэрофлот-росс.авиалин(ПАО)ао,RU0009062285,1,1300,"Публичное акционерное общество ""Аэрофлот – рос...",7712040126,29063984,1-01-00010-A,common_share,stock_shares,TQBR,TQBR
7,212232577,AFM0,AFLT-6.20,None,Фьючерсный контракт AFLT-6.20,None,1,1300,"Публичное акционерное общество ""Аэрофлот – рос...",7712040126,29063984,None,futures,futures_forts,RFUD,None
8,85916,afmc,"АО ""Агрофирма Мценская""",1-01-40534-A,"Акционерное общество ""Агрофирма Мценская""",RU000A0JRAA9,1,486951,"Акционерное общество ""Агрофирма Мценская""",5717060059,None,1-01-40534-A,common_share,stock_shares,MXBD,None
9,76079,AGRO,AGRO-гдр,None,ГДР ROS AGRO PLC ORD SHS,US7496552057,1,11660,Bank of New York Mellon Corporation (Глобальны...,0000003817,None,None,depositary_receipt,stock_dr,TQBR,TQBR


Для информации, значения основных справочников Мосбиржи можно увидеть тут (https://iss.moex.com/iss/index.xml)

Определения справочников (как на сайте):
engine - торговая система.
markets - рынки
boards - режим торгов

engine
<row id="1" name="stock" title="Фондовый рынок и рынок депозитов"/>
<row id="2" name="state" title="Рынок ГЦБ (размещение)"/>
<row id="3" name="currency" title="Валютный рынок"/>
<row id="4" name="futures" title="Срочный рынок"/>
<row id="5" name="commodity" title="Товарный рынок"/>
<row id="6" name="interventions" title="Товарные интервенции"/>
<row id="7" name="offboard" title="ОТС-система"/>

markets
<row id="5" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="index" market_title="Индексы фондового рынка" market_id="5" marketplace="INDICES"/>
<row id="1" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="shares" market_title="Рынок акций" market_id="1" marketplace="MXSE"/>
<row id="2" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="bonds" market_title="Рынок облигаций" market_id="2" marketplace="MXSE"/>
<row id="4" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="ndm" market_title="Режим переговорных сделок" market_id="4" marketplace=""/>
<row id="29" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="otc" market_title="ОТС" market_id="29" marketplace=""/>
<row id="27" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="ccp" market_title="РЕПО с ЦК" market_id="27" marketplace="MXSE"/>
<row id="35" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="deposit" market_title="Депозиты с ЦК" market_id="35" marketplace=""/>
<row id="3" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="repo" market_title="Рынок сделок РЕПО" market_id="3" marketplace=""/>
<row id="28" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="qnv" market_title="Квал. инвесторы" market_id="28" marketplace=""/>
<row id="36" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="mamc" market_title="Мультивалютный рынок смешанных активов" market_id="36" marketplace=""/>
<row id="33" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="moexboard" market_title="MOEX Board" market_id="33" marketplace=""/>
<row id="21" trade_engine_id="3" trade_engine_name="currency" trade_engine_title="Валютный рынок" market_name="basket" market_title="Бивалютная корзина" market_id="21" marketplace=""/>
<row id="10" trade_engine_id="3" trade_engine_name="currency" trade_engine_title="Валютный рынок" market_name="selt" market_title="Биржевые сделки с ЦК" market_id="10" marketplace="MXCX"/>
<row id="34" trade_engine_id="3" trade_engine_name="currency" trade_engine_title="Валютный рынок" market_name="futures" market_title="Поставочные фьючерсы" market_id="34" marketplace="MXCX"/>
<row id="41" trade_engine_id="3" trade_engine_name="currency" trade_engine_title="Валютный рынок" market_name="index" market_title="Валютный фиксинг" market_id="41" marketplace="FIXING"/>
<row id="12" trade_engine_id="4" trade_engine_name="futures" trade_engine_title="Срочный рынок" market_name="main" market_title="Срочные инструменты" market_id="12" marketplace=""/>
<row id="22" trade_engine_id="4" trade_engine_name="futures" trade_engine_title="Срочный рынок" market_name="forts" market_title="ФОРТС" market_id="22" marketplace="FORTS"/>
<row id="24" trade_engine_id="4" trade_engine_name="futures" trade_engine_title="Срочный рынок" market_name="options" market_title="Опционы ФОРТС" market_id="24" marketplace="OPTIONS"/>
<row id="37" trade_engine_id="4" trade_engine_name="futures" trade_engine_title="Срочный рынок" market_name="fortsiqs" market_title="Фьючерсы IQS" market_id="37" marketplace=""/>
<row id="38" trade_engine_id="4" trade_engine_name="futures" trade_engine_title="Срочный рынок" market_name="optionsiqs" market_title="Опционы IQS" market_id="38" marketplace=""/>
<row id="9" trade_engine_id="2" trade_engine_name="state" trade_engine_title="Рынок ГЦБ (размещение)" market_name="index" market_title="Индексы ГКО/ОФЗ" market_id="9" marketplace=""/>
<row id="6" trade_engine_id="2" trade_engine_name="state" trade_engine_title="Рынок ГЦБ (размещение)" market_name="bonds" market_title="Облигации ГЦБ" market_id="6" marketplace=""/>
<row id="7" trade_engine_id="2" trade_engine_name="state" trade_engine_title="Рынок ГЦБ (размещение)" market_name="repo" market_title="Междилерское РЕПО" market_id="7" marketplace=""/>
<row id="8" trade_engine_id="2" trade_engine_name="state" trade_engine_title="Рынок ГЦБ (размещение)" market_name="ndm" market_title="Внесистемные сделки" market_id="8" marketplace=""/>
<row id="39" trade_engine_id="7" trade_engine_name="offboard" trade_engine_title="ОТС-система" market_name="bonds" market_title="Облигации" market_id="39" marketplace=""/>
<row id="18" trade_engine_id="5" trade_engine_name="commodity" trade_engine_title="Товарный рынок" market_name="futures" market_title="Секция стандартных контрактов АО НТБ" market_id="18" marketplace=""/>
<row id="20" trade_engine_id="6" trade_engine_name="interventions" trade_engine_title="Товарные интервенции" market_name="grain" market_title="Интервенции по зерну" market_id="20" marketplace=""/>
<row id="23" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="standard" market_title="Standard" market_id="23" marketplace=""/>
<row id="25" trade_engine_id="1" trade_engine_name="stock" trade_engine_title="Фондовый рынок и рынок депозитов" market_name="classica" market_title="Classica" market_id="25" marketplace=""/>

boards
<row id="260" board_group_id="99" engine_id="1" market_id="1" boardid="EQTD" board_title="Т0 ETF (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="177" board_group_id="57" engine_id="1" market_id="1" boardid="TQIF" board_title="Т+: Паи - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="178" board_group_id="57" engine_id="1" market_id="1" boardid="TQTF" board_title="Т+: ETF - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="129" board_group_id="57" engine_id="1" market_id="1" boardid="TQBR" board_title="Т+: Акции и ДР - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="130" board_group_id="57" engine_id="1" market_id="1" boardid="TQBS" board_title="Т+: А2-Акции и паи - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="131" board_group_id="57" engine_id="1" market_id="1" boardid="TQNL" board_title="Т+: Б-Акции и паи - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="132" board_group_id="57" engine_id="1" market_id="1" boardid="TQLV" board_title="Т+: В-Акции и ДР - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="133" board_group_id="57" engine_id="1" market_id="1" boardid="TQLI" board_title="Т+: И-Акции - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="134" board_group_id="57" engine_id="1" market_id="1" boardid="TQNE" board_title="Т+: Акции, паи и ДР внесписочные - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="224" board_group_id="57" engine_id="1" market_id="1" boardid="TQBD" board_title="Т+: Акции и ДР (расч. в USD) - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="158" board_group_id="57" engine_id="1" market_id="1" boardid="TQDE" board_title="Т+: Акции Д - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="225" board_group_id="156" engine_id="1" market_id="1" boardid="TQTD" board_title="Т+: ETF (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="1" board_group_id="6" engine_id="1" market_id="1" boardid="EQBR" board_title="Основной режим: А1-Акции и паи - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="2" board_group_id="6" engine_id="1" market_id="1" boardid="EQBS" board_title="Основной режим: А2-Акции и паи - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="4" board_group_id="6" engine_id="1" market_id="1" boardid="EQNL" board_title="Основной режим: Б-Акции и паи - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="3" board_group_id="6" engine_id="1" market_id="1" boardid="EQLV" board_title="Основной режим: В-Акции и РДР - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="117" board_group_id="6" engine_id="1" market_id="1" boardid="EQDE" board_title="Основной режим: Акции Д - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="28" board_group_id="6" engine_id="1" market_id="1" boardid="EQLI" board_title="Основной режим: И-Акции - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="5" board_group_id="6" engine_id="1" market_id="1" boardid="EQNE" board_title="Основной режим: Акции и паи внесписочные - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="281" board_group_id="125" engine_id="1" market_id="1" boardid="SPEQ" board_title="Поставка по СК (акции)" is_traded="1" has_candles="1" is_primary="0"/>
<row id="36" board_group_id="21" engine_id="1" market_id="1" boardid="SMAL" board_title="Т+: Неполные лоты (акции) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="305" board_group_id="42" engine_id="1" market_id="1" boardid="TQDP" board_title="Крупные пакеты - Акции - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="100" board_group_id="42" engine_id="1" market_id="1" boardid="EQDP" board_title="Крупные пакеты - Акции - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="313" board_group_id="157" engine_id="1" market_id="1" boardid="TQBE" board_title="Т+: Акции и ДР (расч. в EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="314" board_group_id="158" engine_id="1" market_id="1" boardid="TQTE" board_title="Т+: ETF (расч. в EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="315" board_group_id="159" engine_id="1" market_id="1" boardid="EQTU" board_title="Т0 ETF (расч. в EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="95" board_group_id="30" engine_id="1" market_id="1" boardid="EQCC" board_title="ЦК - режим основных торгов - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="226" board_group_id="193" engine_id="1" market_id="2" boardid="TQOD" board_title="Т+: Облигации (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="135" board_group_id="58" engine_id="1" market_id="2" boardid="TQOB" board_title="Т+: Гособлигации - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="136" board_group_id="58" engine_id="1" market_id="2" boardid="TQOS" board_title="Т+: А2-Облигации - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="137" board_group_id="58" engine_id="1" market_id="2" boardid="TQNO" board_title="Т+: Б-Облигации - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="138" board_group_id="58" engine_id="1" market_id="2" boardid="TQOV" board_title="Т+: В-Облигации - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="139" board_group_id="58" engine_id="1" market_id="2" boardid="TQNB" board_title="Т+: Облигации внесписочные - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="140" board_group_id="58" engine_id="1" market_id="2" boardid="TQUS" board_title="Т+: Облигации внеспис. в ин.валюте - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="349" board_group_id="58" engine_id="1" market_id="2" boardid="TQCB" board_title="Т+: Облигации- безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="179" board_group_id="68" engine_id="1" market_id="2" boardid="TQTC" board_title="T+: ETC - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="8" board_group_id="7" engine_id="1" market_id="2" boardid="EQOB" board_title="Т0 Облигации - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="9" board_group_id="7" engine_id="1" market_id="2" boardid="EQOS" board_title="Основной режим: А2-Облигации - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="7" board_group_id="7" engine_id="1" market_id="2" boardid="EQNO" board_title="Основной режим: Б-Облигации - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="29" board_group_id="7" engine_id="1" market_id="2" boardid="EQOV" board_title="Основной режим: В-Облигации - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="93" board_group_id="7" engine_id="1" market_id="2" boardid="EQDB" board_title="Основной режим: Облигации Д - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="6" board_group_id="7" engine_id="1" market_id="2" boardid="EQNB" board_title="Основной режим: Облигации внесписочные - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="88" board_group_id="7" engine_id="1" market_id="2" boardid="EQUS" board_title="Основной режим: Облигации внеспис. в ин.валюте - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="165" board_group_id="7" engine_id="1" market_id="2" boardid="EQEO" board_title="Основной режим: Облигации (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="119" board_group_id="105" engine_id="1" market_id="2" boardid="SPOB" board_title="Поставка по ОФЗ" is_traded="1" has_candles="1" is_primary="0"/>
<row id="176" board_group_id="69" engine_id="1" market_id="2" boardid="EQTC" board_title="T0 ETC - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="37" board_group_id="67" engine_id="1" market_id="2" boardid="EQEU" board_title="Облигации (USD) - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="228" board_group_id="67" engine_id="1" market_id="2" boardid="EQGO" board_title="Облигации (GBP) - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="229" board_group_id="67" engine_id="1" market_id="2" boardid="EQYO" board_title="Облигации (CNY) - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="122" board_group_id="122" engine_id="1" market_id="2" boardid="AUBB" board_title="Выкуп: Аукцион - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="121" board_group_id="123" engine_id="1" market_id="2" boardid="AUCT" board_title="Размещение: Аукцион - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="227" board_group_id="77" engine_id="1" market_id="2" boardid="TQDB" board_title="Крупные пакеты: Облигации - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="323" board_group_id="167" engine_id="1" market_id="2" boardid="TQDU" board_title="Крупные пакеты – Облигации (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="24" board_group_id="201" engine_id="1" market_id="3" boardid="RPMO" board_title="РЕПО-М - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="34" board_group_id="1" engine_id="1" market_id="3" boardid="RPEQ" board_title="РЕПО: Акции и паи - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="23" board_group_id="95" engine_id="1" market_id="3" boardid="RPMA" board_title="РЕПО c акциями - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="35" board_group_id="94" engine_id="1" market_id="3" boardid="RPEU" board_title="РЕПО в ин. валюте (USD) - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="162" board_group_id="94" engine_id="1" market_id="3" boardid="RPUA" board_title="РЕПО c акциями (USD) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="163" board_group_id="94" engine_id="1" market_id="3" boardid="RPUO" board_title="РЕПО c облигациями (USD) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="164" board_group_id="90" engine_id="1" market_id="3" boardid="RPEO" board_title="РЕПО в ин. валюте (EUR) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="254" board_group_id="2" engine_id="1" market_id="3" boardid="RPEY" board_title="РЕПО в ин.валюте (CNY) - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="238" board_group_id="2" engine_id="1" market_id="3" boardid="RPGO" board_title="РЕПО c облигациями (GBP) - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="96" board_group_id="31" engine_id="1" market_id="3" boardid="RPCC" board_title="РЕПО с ЦК - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="173" board_group_id="59" engine_id="1" market_id="4" boardid="PTIF" board_title="РПС с ЦК: Паи - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="174" board_group_id="59" engine_id="1" market_id="4" boardid="PTTF" board_title="РПС с ЦК: ETF - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="175" board_group_id="59" engine_id="1" market_id="4" boardid="PTTC" board_title="РПС с ЦК: ETC - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="142" board_group_id="59" engine_id="1" market_id="4" boardid="PTEQ" board_title="РПС с ЦК: Акции и ДР - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="143" board_group_id="59" engine_id="1" market_id="4" boardid="PTES" board_title="РПС С ЦК: А2-Акции и паи - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="144" board_group_id="59" engine_id="1" market_id="4" boardid="PTNL" board_title="РПС С ЦК: Б-Акции и паи - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="145" board_group_id="59" engine_id="1" market_id="4" boardid="PTLV" board_title="РПС С ЦК: В-Акции и ДР - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="146" board_group_id="59" engine_id="1" market_id="4" boardid="PTLI" board_title="РПС С ЦК: И-Акции - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="147" board_group_id="59" engine_id="1" market_id="4" boardid="PTNE" board_title="РПС С ЦК: Акции, паи и ДР внесписочные - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="148" board_group_id="59" engine_id="1" market_id="4" boardid="PTOB" board_title="РПС с ЦК: Облигации - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="149" board_group_id="59" engine_id="1" market_id="4" boardid="PTOS" board_title="РПС С ЦК: А2-Облигации - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="150" board_group_id="59" engine_id="1" market_id="4" boardid="PTNO" board_title="РПС С ЦК: Б-Облигации - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="151" board_group_id="59" engine_id="1" market_id="4" boardid="PTOV" board_title="РПС С ЦК: В-Облигации - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="152" board_group_id="59" engine_id="1" market_id="4" boardid="PTNB" board_title="РПС С ЦК: Облигации внесписочные - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="153" board_group_id="59" engine_id="1" market_id="4" boardid="PTUS" board_title="РПС С ЦК: Облигации в ин.валюте - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="154" board_group_id="59" engine_id="1" market_id="4" boardid="PTEU" board_title="РПС с ЦК: Облигации (USD) - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="161" board_group_id="59" engine_id="1" market_id="4" boardid="PTDE" board_title="РПС с ЦК: Акции Д - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="233" board_group_id="74" engine_id="1" market_id="4" boardid="PSSD" board_title="РПС: Акции и ДР (USD) - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="234" board_group_id="74" engine_id="1" market_id="4" boardid="PSTD" board_title="РПС: ETF (USD) - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="170" board_group_id="8" engine_id="1" market_id="4" boardid="PSIF" board_title="РПС: Паи - адрес." is_traded="1" has_candles="0" is_primary="1"/>
<row id="171" board_group_id="8" engine_id="1" market_id="4" boardid="PSTF" board_title="РПС: ETF - адрес." is_traded="1" has_candles="0" is_primary="1"/>
<row id="172" board_group_id="8" engine_id="1" market_id="4" boardid="PSTC" board_title="РПС: ETC - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="11" board_group_id="8" engine_id="1" market_id="4" boardid="PSEQ" board_title="РПС: Акции - адрес." is_traded="1" has_candles="0" is_primary="1"/>
<row id="12" board_group_id="8" engine_id="1" market_id="4" boardid="PSES" board_title="РПС: А2-Акции и паи - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="15" board_group_id="8" engine_id="1" market_id="4" boardid="PSNL" board_title="РПС: Б-Акции и паи - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="32" board_group_id="8" engine_id="1" market_id="4" boardid="PSLV" board_title="РПС: В-Акции и ДР - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="118" board_group_id="8" engine_id="1" market_id="4" boardid="PSDE" board_title="РПС: Акции Д - адрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="31" board_group_id="8" engine_id="1" market_id="4" boardid="PSLI" board_title="РПС: И-Акции - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="14" board_group_id="8" engine_id="1" market_id="4" boardid="PSNE" board_title="РПС: Акции, паи и ДР внесписочные - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="17" board_group_id="8" engine_id="1" market_id="4" boardid="PSOB" board_title="РПС: Облигации - адрес." is_traded="1" has_candles="0" is_primary="1"/>
<row id="18" board_group_id="8" engine_id="1" market_id="4" boardid="PSOS" board_title="РПС: А2-Облигации - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="16" board_group_id="8" engine_id="1" market_id="4" boardid="PSNO" board_title="РПС: Б-Облигации - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="33" board_group_id="8" engine_id="1" market_id="4" boardid="PSOV" board_title="РПС: В-Облигации - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="92" board_group_id="8" engine_id="1" market_id="4" boardid="PSDB" board_title="РПС: Облигации Д - адрес." is_traded="1" has_candles="0" is_primary="1"/>
<row id="13" board_group_id="8" engine_id="1" market_id="4" boardid="PSNB" board_title="РПС: Облигации внесписочные - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="89" board_group_id="8" engine_id="1" market_id="4" boardid="PSUS" board_title="РПС: Облигации в ин.валюте - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="236" board_group_id="8" engine_id="1" market_id="4" boardid="PSYO" board_title="РПС: Облигации (CNY) - адрес." is_traded="0" has_candles="0" is_primary="1"/>
<row id="30" board_group_id="106" engine_id="1" market_id="4" boardid="PSEU" board_title="РПС: Облигации (USD) - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="160" board_group_id="106" engine_id="1" market_id="4" boardid="PSEO" board_title="РПС: Облигации (EUR) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="235" board_group_id="106" engine_id="1" market_id="4" boardid="PSGO" board_title="РПС: Облигации (GBP) - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="98" board_group_id="107" engine_id="1" market_id="4" boardid="PSAU" board_title="Размещение - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="99" board_group_id="107" engine_id="1" market_id="4" boardid="PAUS" board_title="Размещение (USD) - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="159" board_group_id="107" engine_id="1" market_id="4" boardid="PAEU" board_title="Размещение (EUR) - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="169" board_group_id="107" engine_id="1" market_id="4" boardid="PAGB" board_title="Размещение (GBP) - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="237" board_group_id="107" engine_id="1" market_id="4" boardid="PACY" board_title="Размещение (CNY) - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="120" board_group_id="108" engine_id="1" market_id="4" boardid="PSBB" board_title="Выкуп - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="278" board_group_id="121" engine_id="1" market_id="4" boardid="PSBE" board_title="Выкуп (EUR) - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="282" board_group_id="126" engine_id="1" market_id="4" boardid="OTCB" board_title="Анонимный РПС - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="353" board_group_id="203" engine_id="1" market_id="4" boardid="OTCU" board_title="Анонимный РПС (USD) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="355" board_group_id="205" engine_id="1" market_id="4" boardid="OTCE" board_title="Анонимный РПС (EUR) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="230" board_group_id="73" engine_id="1" market_id="4" boardid="PTSD" board_title="РПС с ЦК: Акции и ДР (USD) - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="231" board_group_id="73" engine_id="1" market_id="4" boardid="PTTD" board_title="РПС с ЦК: ETF (USD) - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="232" board_group_id="73" engine_id="1" market_id="4" boardid="PTOD" board_title="РПС с ЦК: Облигации (USD) - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="277" board_group_id="124" engine_id="1" market_id="4" boardid="PSBU" board_title="Выкуп (USD) - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="316" board_group_id="160" engine_id="1" market_id="4" boardid="PSSE" board_title="РПС: Акции и ДР (расч. в EUR) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="317" board_group_id="161" engine_id="1" market_id="4" boardid="PSTE" board_title="РПС: ETF (расч. в EUR) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="318" board_group_id="162" engine_id="1" market_id="4" boardid="PTSE" board_title="РПС с ЦК: Акции и ДР (расч. в EUR) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="319" board_group_id="163" engine_id="1" market_id="4" boardid="PTTE" board_title="РПС с ЦК: ETF (расч. в EUR) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="97" board_group_id="32" engine_id="1" market_id="4" boardid="PSCC" board_title="РПС с ЦК - адрес." is_traded="0" has_candles="0" is_primary="0"/>
<row id="44" board_group_id="9" engine_id="1" market_id="5" boardid="SNDX" board_title="Индексы фондового рынка" is_traded="1" has_candles="1" is_primary="1"/>
<row id="102" board_group_id="9" engine_id="1" market_id="5" boardid="RTSI" board_title="Индексы РТС" is_traded="1" has_candles="1" is_primary="1"/>
<row id="265" board_group_id="104" engine_id="1" market_id="5" boardid="INAV" board_title="INAV" is_traded="1" has_candles="1" is_primary="0"/>
<row id="312" board_group_id="155" engine_id="1" market_id="5" boardid="MMIX" board_title="Money Market IndeX" is_traded="1" has_candles="1" is_primary="0"/>
<row id="104" board_group_id="36" engine_id="1" market_id="23" boardid="STMR" board_title="Standard: дневная сессия - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="105" board_group_id="36" engine_id="1" market_id="23" boardid="SDMR" board_title="Standard: вечерняя сессия - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="108" board_group_id="41" engine_id="1" market_id="23" boardid="STAD" board_title="Standard: дневная сессия - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="109" board_group_id="41" engine_id="1" market_id="23" boardid="SDAD" board_title="Standard: вечерняя сессия - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="106" board_group_id="40" engine_id="1" market_id="23" boardid="STRP" board_title="Standard: сделки репо, дневная сессия - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="107" board_group_id="40" engine_id="1" market_id="23" boardid="SDRP" board_title="Standard: сделки репо, вечерняя сессия - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="112" board_group_id="43" engine_id="1" market_id="25" boardid="CLMR" board_title="Classica - безадрес." is_traded="0" has_candles="1" is_primary="1"/>
<row id="113" board_group_id="44" engine_id="1" market_id="25" boardid="CLAD" board_title="Classica - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="239" board_group_id="92" engine_id="1" market_id="27" boardid="EQRD" board_title="РЕПО с ЦК 1 день (USD) - безадрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="240" board_group_id="93" engine_id="1" market_id="27" boardid="EQRE" board_title="РЕПО с ЦК 1 день (EUR) - безадрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="241" board_group_id="76" engine_id="1" market_id="27" boardid="EQWP" board_title="РЕПО с ЦК 7 дн. - безадрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="245" board_group_id="78" engine_id="1" market_id="27" boardid="EQWD" board_title="РЕПО с ЦК 7 дней (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="246" board_group_id="79" engine_id="1" market_id="27" boardid="EQWE" board_title="РЕПО с ЦК 7 дней (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="123" board_group_id="50" engine_id="1" market_id="27" boardid="EQRP" board_title="РЕПО с ЦК 1 день - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="298" board_group_id="142" engine_id="1" market_id="27" boardid="EQMP" board_title="РЕПО с ЦК 1 мес.(RUB) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="299" board_group_id="143" engine_id="1" market_id="27" boardid="EQMD" board_title="РЕПО с ЦК 1 мес. (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="300" board_group_id="144" engine_id="1" market_id="27" boardid="EQME" board_title="РЕПО с ЦК 1 мес. (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="301" board_group_id="145" engine_id="1" market_id="27" boardid="EQTP" board_title="РЕПО с ЦК 3 мес. (RUB) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="302" board_group_id="146" engine_id="1" market_id="27" boardid="ETQD" board_title="РЕПО с ЦК 3 мес. (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="303" board_group_id="147" engine_id="1" market_id="27" boardid="EQTE" board_title="РЕПО с ЦК 3 мес. (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="304" board_group_id="148" engine_id="1" market_id="27" boardid="LIQR" board_title="РЕПО с ЦК: Урегулирование - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="309" board_group_id="152" engine_id="1" market_id="27" boardid="EQRY" board_title="РЕПО с ЦК 1 день (CNY) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="310" board_group_id="153" engine_id="1" market_id="27" boardid="PSRY" board_title="РЕПО с ЦК (CNY) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="125" board_group_id="65" engine_id="1" market_id="27" boardid="PSRP" board_title="РЕПО с ЦК - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="242" board_group_id="91" engine_id="1" market_id="27" boardid="PSRD" board_title="РЕПО с ЦК (USD) - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="243" board_group_id="96" engine_id="1" market_id="27" boardid="PSRE" board_title="РЕПО с ЦК (EUR) - адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="253" board_group_id="86" engine_id="1" market_id="27" boardid="GCRP" board_title="РЕПО с ЦК с КСУ 1 день - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="250" board_group_id="83" engine_id="1" market_id="27" boardid="GCOW" board_title="РЕПО с ЦК с КСУ 7 дн. - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="247" board_group_id="80" engine_id="1" market_id="27" boardid="GCSW" board_title="РЕПО с ЦК с КСУ 14 дн. - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="251" board_group_id="84" engine_id="1" market_id="27" boardid="GCOM" board_title="РЕПО с ЦК с КСУ 1 месяц - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="248" board_group_id="81" engine_id="1" market_id="27" boardid="GCSM" board_title="РЕПО с ЦК с КСУ 2 месяца - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="252" board_group_id="85" engine_id="1" market_id="27" boardid="GCTM" board_title="РЕПО с ЦК с КСУ 3 месяца - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="264" board_group_id="103" engine_id="1" market_id="27" boardid="GCUM" board_title="РЕПО с ЦК с КСУ 6 мес. - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="263" board_group_id="102" engine_id="1" market_id="27" boardid="GCOY" board_title="РЕПО с ЦК с КСУ 1 год - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="249" board_group_id="82" engine_id="1" market_id="27" boardid="PSGC" board_title="РЕПО с ЦК с КСУ - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="268" board_group_id="111" engine_id="1" market_id="27" boardid="GURP" board_title="РЕПО с ЦК с КСУ 1 день (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="269" board_group_id="112" engine_id="1" market_id="27" boardid="GUOW" board_title="РЕПО с ЦК с КСУ 7 дн. (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="270" board_group_id="113" engine_id="1" market_id="27" boardid="GUSW" board_title="РЕПО с ЦК с КСУ 14 дн. (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="271" board_group_id="114" engine_id="1" market_id="27" boardid="GUOM" board_title="РЕПО с ЦК с КСУ 1 месяц (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="272" board_group_id="115" engine_id="1" market_id="27" boardid="GUSM" board_title="РЕПО с ЦК с КСУ 2 месяца (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="273" board_group_id="116" engine_id="1" market_id="27" boardid="GUTM" board_title="РЕПО с ЦК с КСУ 3 месяца (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="274" board_group_id="117" engine_id="1" market_id="27" boardid="GUUM" board_title="РЕПО с ЦК с КСУ 6 месяцев (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="275" board_group_id="118" engine_id="1" market_id="27" boardid="GUOY" board_title="РЕПО с ЦК с КСУ 1 год (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="276" board_group_id="119" engine_id="1" market_id="27" boardid="PUGC" board_title="РЕПО с ЦК с КСУ (USD) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="283" board_group_id="127" engine_id="1" market_id="27" boardid="GERP" board_title="РЕПО с ЦК с КСУ 1 день (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="284" board_group_id="128" engine_id="1" market_id="27" boardid="GEOW" board_title="РЕПО с ЦК с КСУ 7 дн. (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="285" board_group_id="129" engine_id="1" market_id="27" boardid="GESW" board_title="РЕПО с ЦК с КСУ 14 дн. (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="286" board_group_id="130" engine_id="1" market_id="27" boardid="GEOM" board_title="РЕПО с ЦК с КСУ 1 месяц (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="287" board_group_id="131" engine_id="1" market_id="27" boardid="GESM" board_title="РЕПО с ЦК с КСУ 2 месяца (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="288" board_group_id="132" engine_id="1" market_id="27" boardid="GETM" board_title="РЕПО с ЦК с КСУ 3 месяца (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="289" board_group_id="133" engine_id="1" market_id="27" boardid="GEUM" board_title="РЕПО с ЦК с КСУ 6 месяцев (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="290" board_group_id="134" engine_id="1" market_id="27" boardid="GEOY" board_title="РЕПО с ЦК с КСУ 1 год (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="291" board_group_id="135" engine_id="1" market_id="27" boardid="PEGC" board_title="РЕПО с ЦК с КСУ адресн.(EUR) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="325" board_group_id="169" engine_id="1" market_id="27" boardid="FKRP" board_title="Аукцион с ЦК 1 день - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="327" board_group_id="171" engine_id="1" market_id="27" boardid="FKOW" board_title="Аукцион с ЦК 1 неделя - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="329" board_group_id="173" engine_id="1" market_id="27" boardid="FKSW" board_title="Аукцион с ЦК 2 недели - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="331" board_group_id="175" engine_id="1" market_id="27" boardid="FKOM" board_title="Аукцион с ЦК 1 месяц - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="333" board_group_id="177" engine_id="1" market_id="27" boardid="FKSM" board_title="Аукцион с ЦК 2 месяца - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="335" board_group_id="179" engine_id="1" market_id="27" boardid="FKTM" board_title="Аукцион с ЦК 3 месяца - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="337" board_group_id="181" engine_id="1" market_id="27" boardid="FKUM" board_title="Аукцион с ЦК 6 месяцев - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="339" board_group_id="183" engine_id="1" market_id="27" boardid="FKOY" board_title="Аукцион с ЦК 1 год - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="343" board_group_id="187" engine_id="1" market_id="27" boardid="GSNM" board_title="РЕПО с ЦК с КСУ 9 месяцев - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="345" board_group_id="189" engine_id="1" market_id="27" boardid="GUNM" board_title="РЕПО с ЦК с КСУ 9 мес. (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="347" board_group_id="191" engine_id="1" market_id="27" boardid="GENM" board_title="РЕПО с ЦК с КСУ 9 мес. (EUR) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="167" board_group_id="63" engine_id="1" market_id="28" boardid="TQQI" board_title="Т+: Для квал. инвесторов - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="155" board_group_id="60" engine_id="1" market_id="28" boardid="EQQI" board_title="Основной режим: Для квал. инвесторов - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="156" board_group_id="61" engine_id="1" market_id="28" boardid="PSQI" board_title="РПС: Для квал. инвесторов - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="168" board_group_id="64" engine_id="1" market_id="28" boardid="PTQI" board_title="РПС с ЦК: Для квал. инвесторов - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="157" board_group_id="62" engine_id="1" market_id="28" boardid="RPQI" board_title="РЕПО: Для квал. инвесторов - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="166" board_group_id="62" engine_id="1" market_id="28" boardid="RPUQ" board_title="РЕПО: Для квал. инвесторов (USD) - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="255" board_group_id="87" engine_id="1" market_id="28" boardid="TQQD" board_title="Т+: Для квал. инвесторов (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="183" board_group_id="71" engine_id="1" market_id="29" boardid="SOTC" board_title="Внебиржевые сделки" is_traded="1" has_candles="0" is_primary="1"/>
<row id="244" board_group_id="72" engine_id="1" market_id="33" boardid="MXBD" board_title="MOEX Board" is_traded="1" has_candles="0" is_primary="1"/>
<row id="258" board_group_id="97" engine_id="1" market_id="35" boardid="TDEP" board_title="Депозиты с ЦК - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="259" board_group_id="98" engine_id="1" market_id="35" boardid="NDEP" board_title="Депозиты с ЦК - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="266" board_group_id="109" engine_id="1" market_id="35" boardid="NDPU" board_title="Депозиты с ЦК (USD) - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="292" board_group_id="136" engine_id="1" market_id="35" boardid="NDPE" board_title="Депозиты с ЦК (EUR) - адрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="267" board_group_id="110" engine_id="1" market_id="35" boardid="TDPU" board_title="Депозиты с ЦК (USD) - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="293" board_group_id="137" engine_id="1" market_id="35" boardid="TDPE" board_title="Депозиты с ЦК (EUR) - безадрес." is_traded="0" has_candles="1" is_primary="0"/>
<row id="341" board_group_id="185" engine_id="1" market_id="35" boardid="ADEP" board_title="Депозиты с ЦК - Аукцион" is_traded="1" has_candles="1" is_primary="0"/>
<row id="279" board_group_id="120" engine_id="1" market_id="36" boardid="LIQB" board_title="Продажа обеспечения бирж.рынок - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="19" board_group_id="10" engine_id="2" market_id="6" boardid="MAIN" board_title="ГЦБ" is_traded="0" has_candles="1" is_primary="1"/>
<row id="25" board_group_id="3" engine_id="2" market_id="7" boardid="RPDD" board_title="Сделки междилерского РЕПО" is_traded="0" has_candles="0" is_primary="1"/>
<row id="20" board_group_id="11" engine_id="2" market_id="8" boardid="NEGD" board_title="ГЦБ: Внесистемные сделки" is_traded="0" has_candles="0" is_primary="1"/>
<row id="51" board_group_id="12" engine_id="2" market_id="9" boardid="GNDX" board_title="Индексы ГЦБ" is_traded="0" has_candles="1" is_primary="1"/>
<row id="21" board_group_id="13" engine_id="3" market_id="10" boardid="CETS" board_title="Системные сделки - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="351" board_group_id="13" engine_id="3" market_id="10" boardid="SDBP" board_title="Крупные сделки - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="52" board_group_id="13" engine_id="3" market_id="10" boardid="CURR" board_title="Дневная сессия" is_traded="0" has_candles="1" is_primary="1"/>
<row id="116" board_group_id="46" engine_id="3" market_id="10" boardid="CNGD" board_title="Внесистемные сделки- адрес." is_traded="1" has_candles="0" is_primary="0"/>
<row id="308" board_group_id="151" engine_id="3" market_id="10" boardid="LICU" board_title="Внесистемные сделки урегулирования - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="261" board_group_id="100" engine_id="3" market_id="10" boardid="FIXS" board_title="Фиксинг системный - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="262" board_group_id="101" engine_id="3" market_id="10" boardid="FIXN" board_title="Фиксинг внесистемный- адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="306" board_group_id="149" engine_id="3" market_id="10" boardid="WAPS" board_title="Системные средневзвешенные - безадрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="307" board_group_id="150" engine_id="3" market_id="10" boardid="WAPN" board_title="Внесистемные средневзвешенные - адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="296" board_group_id="140" engine_id="3" market_id="10" boardid="SPEC" board_title="Поставка - безадресные" is_traded="1" has_candles="1" is_primary="0"/>
<row id="86" board_group_id="26" engine_id="3" market_id="21" boardid="BKT" board_title="Бивалютная корзина" is_traded="1" has_candles="1" is_primary="1"/>
<row id="256" board_group_id="88" engine_id="3" market_id="34" boardid="FUTS" board_title="Фьючерсы системные - безадрес." is_traded="1" has_candles="1" is_primary="1"/>
<row id="257" board_group_id="89" engine_id="3" market_id="34" boardid="FUTN" board_title="Фьючерсы внесистемные- адрес." is_traded="1" has_candles="1" is_primary="0"/>
<row id="321" board_group_id="165" engine_id="3" market_id="41" boardid="FIXI" board_title="Валютный фиксинг" is_traded="1" has_candles="1" is_primary="1"/>
<row id="22" board_group_id="15" engine_id="4" market_id="12" boardid="FOB" board_title="Фьючерсы и Опционы" is_traded="0" has_candles="1" is_primary="1"/>
<row id="101" board_group_id="45" engine_id="4" market_id="22" boardid="RFUD" board_title="Фьючерсы" is_traded="1" has_candles="1" is_primary="1"/>
<row id="103" board_group_id="35" engine_id="4" market_id="24" boardid="ROPD" board_title="Опционы" is_traded="1" has_candles="1" is_primary="1"/>
<row id="294" board_group_id="138" engine_id="4" market_id="37" boardid="FIQS" board_title="Фьючерсы IQS" is_traded="1" has_candles="1" is_primary="1"/>
<row id="295" board_group_id="139" engine_id="4" market_id="38" boardid="OIQS" board_title="Опционы IQS" is_traded="1" has_candles="1" is_primary="1"/>
<row id="80" board_group_id="20" engine_id="5" market_id="18" boardid="FOCM" board_title="Фьючерсы на товарные активы" is_traded="0" has_candles="1" is_primary="1"/>
<row id="94" board_group_id="24" engine_id="6" market_id="20" boardid="GSEL" board_title="Интервенции по продаже зерна" is_traded="1" has_candles="0" is_primary="1"/>
<row id="297" board_group_id="141" engine_id="7" market_id="39" boardid="OBBO" board_title="ОТС-система: облигации" is_traded="1" has_candles="1" is_primary="1"/>

Получим спецификацию по инструменту. Попробуем два тикера: Газпром и Индекс московской фондовой биржи.

In [4]:
ticker = 'MOEX'
#ticker = 'GAZP'

response = requests.get(url_prefix + '/iss/securities/'+ ticker + '.json')
data = json.loads(response.text)['description']['data']

ticker_description = pd.DataFrame(data=json.loads(response.text)['description']['data'], 
                                  columns=json.loads(response.text)['description']['columns'])
ticker_boards = pd.DataFrame(data=json.loads(response.text)['boards']['data'], 
                             columns=json.loads(response.text)['boards']['columns'])
ticker_boards.head(20)

,secid,boardid,title,board_group_id,market_id,market,engine_id,engine,is_traded,decimals,history_from,history_till,listed_from,listed_till,is_primary,currencyid
0,MOEX,RPMO,РЕПО-М - адрес.,201,3,repo,1,stock,1,4,2019-04-22,2020-03-03,2019-04-22,2020-03-04,0,RUB
1,MOEX,TQBR,Т+: Акции и ДР - безадрес.,57,1,shares,1,stock,1,2,2013-04-04,2020-03-03,2013-04-04,2020-03-04,1,RUB
2,MOEX,TQLV,Т+: В-Акции и ДР - безадрес.,57,1,shares,1,stock,0,2,2013-03-25,2013-04-03,2013-03-25,2013-04-03,0,RUB
3,MOEX,EQBR,Основной режим: А1-Акции и паи - безадрес.,6,1,shares,1,stock,0,2,2013-04-04,2013-08-30,2013-04-04,2013-08-30,0,RUB
4,MOEX,EQLV,Основной режим: В-Акции и РДР - безадрес.,6,1,shares,1,stock,0,2,2013-02-04,2013-04-03,2013-02-04,2013-04-03,0,RUB
5,MOEX,SPEQ,Поставка по СК (акции),125,1,shares,1,stock,1,2,2018-06-29,2020-03-03,2018-06-29,2020-03-04,0,RUB
6,MOEX,SOTC,Внебиржевые сделки,71,29,otc,1,stock,0,5,2014-02-24,2020-03-03,2013-02-15,2020-03-04,0,None
7,MOEX,SMAL,Т+: Неполные лоты (акции) - безадрес.,21,1,shares,1,stock,1,2,2013-06-14,2020-03-03,2013-06-11,2020-03-04,0,RUB
8,MOEX,TQDP,Крупные пакеты - Акции - безадрес.,42,1,shares,1,stock,1,2,None,None,2019-03-04,2020-03-04,0,RUB
9,MOEX,EQDP,Крупные пакеты - Акции - безадрес.,42,1,shares,1,stock,0,2,2013-05-21,2019-03-01,2013-05-21,2019-03-01,0,RUB


Теперь необходимо получить исторические данные по инструменту. Инициализируем параметры запроса:

In [5]:
engine = 'stock'
market = 'shares'
board = 'TQBR'

Теперь выясним, за какой период доступна история. Это можно сделать либо запросом через API (ниже), либо вытащить инфу из таблицы ticker_boards.

In [6]:
#http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities/MICEXINDEXCF/dates.xml

response = requests.get(url_prefix + 
                        '/iss/history/engines/'+ engine + 
                        '/markets/' + market + 
                        '/boards/' + board + 
                        '/securities/' + ticker + 
                        '/dates.json')
print(json.loads(response.text)['dates']['data'][0])

['2013-04-04', '2020-03-03']


Получаем исторические данные, по различным условиям.

Интервалы графиков определяются следующим справочником:
<row interval="1" duration="60" days="" title="минута" hint="1м"/>
<row interval="10" duration="600" days="" title="10 минут" hint="10м"/>
<row interval="60" duration="3600" days="" title="час" hint="1ч"/>
<row interval="24" duration="86400" days="" title="день" hint="1д"/>
<row interval="7" duration="604800" days="" title="неделя" hint="1н"/>
<row interval="31" duration="2678400" days="" title="месяц" hint="1М"/>
<row interval="4" duration="8035200" days="" title="квартал" hint="1К"/>

In [7]:
#/iss/engines/[engine]/markets/[market]/securities/[security]/candles

from datetime import datetime

params = {'from': '2019-01-01',
         'till': '2020-02-29',
         'interval': '24',
         'start': '0'}
response = requests.get(url_prefix + 
                        '/iss/history/engines/'+ engine + 
                        '/markets/' + market + 
                        '/boards/' + board + 
                        '/securities/' + ticker + 
                        '/candles.json', params)
data=json.loads(response.text)['history']['data']
data_cols = json.loads(response.text)['history']['columns']
history_data = pd.DataFrame(columns=data_cols)
s=0

print(datetime.now())
while len(data)>0:
    params = {'from': '2019-01-01',
             'till': '2020-02-29',
             'interval': '24',
             'start': str(s)}
    response = requests.get(url_prefix + 
                            '/iss/history/engines/'+ engine + 
                            '/markets/' + market + 
                            '/boards/' + board + 
                            '/securities/' + ticker + 
                            '/candles.json', params)
    data=json.loads(response.text)['history']['data']
    if len(data) > 0:
        history_data = history_data.append(pd.DataFrame(data=data, columns=data_cols))
    s = s + 100
    time.sleep(3)
    print(datetime.now())
    
history_data['TRADEDATE'] = pd.to_datetime(history_data.TRADEDATE)
history_data.reset_index(inplace=True)
history_data.drop('index', axis=1, inplace=True)
history_data.head()

2020-03-04 10:34:20.933017
2020-03-04 10:34:23.260251
2020-03-04 10:34:25.749690
2020-03-04 10:34:28.239128
2020-03-04 10:34:30.729017


,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,WAPRICE,CLOSE,VOLUME,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL
0,TQBR,2019-01-03,МосБиржа,MOEX,21049,812691090.8,80.95,80.77,84.49,84.49,82.95,84.49,9797330,82.95,82.95,84.49,812691090.8,812691090.8,812691090.8,None
1,TQBR,2019-01-04,МосБиржа,MOEX,11567,475115006.1,84.50,83.58,84.80,84.00,84.23,84.00,5640400,84.23,84.23,84.00,475115006.1,475115006.1,475115006.1,None
2,TQBR,2019-01-08,МосБиржа,MOEX,14687,572808692.3,84.46,83.33,84.85,83.70,83.85,83.70,6832940,83.83,83.83,83.70,572808692.3,572808692.3,572808692.3,None
3,TQBR,2019-01-09,МосБиржа,MOEX,20791,779476540.8,84.21,83.41,84.27,83.93,83.86,83.93,9295330,83.86,83.86,83.93,779476540.8,779476540.8,779476540.8,None
4,TQBR,2019-01-10,МосБиржа,MOEX,18168,607702672.9,84.06,83.70,84.89,84.70,84.54,84.70,7188760,84.54,84.54,84.70,607702672.9,607702672.9,607702672.9,None


In [9]:
history_data.to_csv(ticker + '_data.csv')